In [86]:
# https://www.census.gov/data/datasets/2019/econ/cbp
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import sys


#### create dataframes for 2018 & 2021

In [87]:
# create dataframe for 2018 & 2021
df_cbp18 = pd.read_csv('cbp18cd.csv')
df_cbp21 = pd.read_csv('cbp21cd.csv', thousands = ',')
df_cbp18.head(5)

,State FIPS,State,116th Congressional District,2017 NAICS Code,NAICS Description,Number of Establishments,Employment,Employment Noise Flag,"1st Quarter Payroll ($1,000)",1st Quarter Payroll Noise Flag,"Annual Payroll ($1,000)",Annual Payroll Noise Flag
0,01,Alabama,1.0,------,Total for all sectors,15921.0,244615.0,G,2492282.0,G,10417508.0,G
1,01,Alabama,1.0,11----,"Agriculture, forestry, fishing and hunting",142.0,1127.0,G,10591.0,G,44855.0,G
2,01,Alabama,1.0,21----,"Mining, quarrying, and oil and gas extraction",34.0,500.0,G,10491.0,H,41785.0,H
3,01,Alabama,1.0,22----,Utilities,56.0,1618.0,G,46260.0,H,151722.0,H
4,01,Alabama,1.0,23----,Construction,1367.0,16277.0,G,194035.0,G,855500.0,G


In [88]:
df_cbp21.columns

Index(['State FIPS', 'State', '118th Congressional District',
       '2017 NAICS Code', 'NAICS Description', ' Number of Establishments ',
       ' Employment ', 'Employment Noise Flag',
       ' 1st Quarter Payroll ($1,000) ', '1st Quarter Payroll Noise Flag',
       ' Annual Payroll ($1,000) ', 'Annual Payroll Noise Flag'],
      dtype='object')

#### Finding un-necessary rows which are not useful for data analytics

In [89]:
# Finding un-necessary rows which are not useful for data analytics
df_cbp18.loc[df_cbp18['NAICS Description'] == 'Total for all sectors'].head()

,State FIPS,State,116th Congressional District,2017 NAICS Code,NAICS Description,Number of Establishments,Employment,Employment Noise Flag,"1st Quarter Payroll ($1,000)",1st Quarter Payroll Noise Flag,"Annual Payroll ($1,000)",Annual Payroll Noise Flag
0,01,Alabama,1.0,------,Total for all sectors,15921.0,244615.0,G,2492282.0,G,10417508.0,G
21,01,Alabama,2.0,------,Total for all sectors,14184.0,216247.0,G,2089913.0,G,8582686.0,G
42,01,Alabama,3.0,------,Total for all sectors,11746.0,187543.0,G,1640393.0,G,6740794.0,G
63,01,Alabama,4.0,------,Total for all sectors,12581.0,191868.0,G,1720901.0,G,7021599.0,G
84,01,Alabama,5.0,------,Total for all sectors,15250.0,270428.0,G,3235343.0,G,13159970.0,G


In [90]:
df_cbp18.columns


Index(['State FIPS', 'State', '116th Congressional District',
       '2017 NAICS Code', 'NAICS Description', ' Number of Establishments ',
       ' Employment ', 'Employment Noise Flag',
       ' 1st Quarter Payroll ($1,000) ', '1st Quarter Payroll Noise Flag',
       ' Annual Payroll ($1,000) ', 'Annual Payroll Noise Flag'],
      dtype='object')

#### removing Unnecessary columns & rows. cleaning of dataframes

In [91]:
# dropping columns & rows. data cleaning
df_cbp18 = ((df_cbp18.drop(['State FIPS', '2017 NAICS Code', 'Employment Noise Flag',' 1st Quarter Payroll ($1,000) ', 
                          '1st Quarter Payroll Noise Flag','Annual Payroll Noise Flag'], axis = 1)
                          .drop(df_cbp18.loc[df_cbp18['NAICS Description'] == 'Total for all sectors'].index, axis=0)
                          .dropna(axis=0)))
df_cbp21 = ((df_cbp21.drop(['State FIPS', '2017 NAICS Code', 'Employment Noise Flag',' 1st Quarter Payroll ($1,000) ', 
                          '1st Quarter Payroll Noise Flag','Annual Payroll Noise Flag'], axis = 1)
                          .drop(df_cbp21.loc[df_cbp21['NAICS Description'] == 'Total for all sectors'].index, axis=0)
                          .dropna(axis=0)))



#### add one column of AverageSalary by multiplying Annual Payroll($1000) with 1000 & dividing it to Employment. For 2018 & 2021

In [92]:
# add one column of AverageSalary by multiplying Annual Payroll($1000) with 1000 & dividing it to Employment. For 2018 
df_cbp18['AverageSalary_18'] = ((df_cbp18[' Annual Payroll ($1,000) ']*1000) / (df_cbp18[' Employment '])).round()
#df_cbp18


In [93]:
# add one column of AverageSalary by multiplying Annual Payroll($1000) with 1000 & dividing it to Employment. For 2021
df_cbp21['AverageSalary_21'] = ((df_cbp21[' Annual Payroll ($1,000) '] * 1000) / df_cbp21[' Employment ']).round()

#### calculate max average salary per state & 116th congressional district for 2018 & 2021. finding number of counties per state

In [94]:
#calculate max average salary per state & 116th congressional district for 2018
CountyMax_18 = df_cbp18.groupby(['State','116th Congressional District']).agg({'AverageSalary_18':np.max}).reset_index()   
CountyMax_18.State.value_counts()

California              53
Texas                   36
New York                27
Florida                 27
Pennsylvania            18
Illinois                18
Ohio                    16
Michigan                14
Georgia                 14
North Carolina          13
New Jersey              12
Virginia                11
Washington              10
Indiana                  9
Arizona                  9
Tennessee                9
Massachusetts            9
Maryland                 8
Minnesota                8
Missouri                 8
Wisconsin                8
South Carolina           7
Colorado                 7
Alabama                  7
Louisiana                6
Kentucky                 6
Oklahoma                 5
Connecticut              5
Oregon                   5
Utah                     4
Nevada                   4
Mississippi              4
Arkansas                 4
Iowa                     4
Kansas                   4
New Mexico               3
West Virginia            3
N

In [95]:
#calculate max average salary per state & 116th congressional district for 2021
CountyMax_21 = df_cbp21.groupby(['State','118th Congressional District']).agg({'AverageSalary_21':np.max}).reset_index()

#### create a subset of max average salary per state & 116th congressional district from dataframe of 2018 & 2021

In [96]:
# create a subset of max average salary per state & 116th congressional district from dataframe of 2018.
df18_merge = CountyMax_18.merge(df_cbp18, on = ['State', '116th Congressional District', 'AverageSalary_18'], how = 'inner')

In [97]:
# create a subset of max average salary per state & 118th congressional district from dataframe of 2021.
df21_merge = CountyMax_21.merge(df_cbp21, on = ['State', '118th Congressional District', 'AverageSalary_21'], how = 'inner')

In [98]:
df18_merge

,State,116th Congressional District,AverageSalary_18,NAICS Description,Number of Establishments,Employment,"Annual Payroll ($1,000)"
0,Alabama,1.0,101567.0,Management of companies and enterprises,105.0,3216.0,326639.0
1,Alabama,2.0,101712.0,Utilities,67.0,2860.0,290897.0
2,Alabama,3.0,78545.0,"Mining, quarrying, and oil and gas extraction",18.0,512.0,40215.0
3,Alabama,4.0,89785.0,Management of companies and enterprises,71.0,1500.0,134677.0
4,Alabama,5.0,93189.0,Utilities,8.0,111.0,10344.0
...,...,...,...,...,...,...,...
432,Wisconsin,5.0,104394.0,"Mining, quarrying, and oil and gas extraction",18.0,963.0,100531.0
433,Wisconsin,6.0,153960.0,"Mining, quarrying, and oil and gas extraction",26.0,3999.0,615688.0
434,Wisconsin,7.0,90396.0,Utilities,96.0,1296.0,117153.0
435,Wisconsin,8.0,98622.0,Utilities,56.0,1851.0,182550.0


#### create subset for california from subset of 2018 & 2021

In [99]:
#create subset for california from subset of 2018
california18 = df18_merge.loc[df18_merge.State == 'California'].drop('116th Congressional District', axis = 1)
california18.head(3)

,State,AverageSalary_18,NAICS Description,Number of Establishments,Employment,"Annual Payroll ($1,000)"
21,California,144577.0,Utilities,55.0,1175.0,169878.0
22,California,152064.0,Information,499.0,7084.0,1077224.0
23,California,124904.0,Utilities,43.0,1066.0,133148.0


In [100]:
california21 = df21_merge.loc[df21_merge.State == 'California'].drop(' Annual Payroll ($1,000) ', axis = 1)
california21.loc[california21['NAICS Description'] == 'Information']

,State,118th Congressional District,AverageSalary_21,NAICS Description,Number of Establishments,Employment
22,California,2.0,197057.0,Information,499.0,6460.0
30,California,10.0,193516.0,Information,418.0,8258.0
31,California,11.0,327267.0,Information,1865.0,91735.0
34,California,14.0,248708.0,Information,351.0,18622.0
35,California,15.0,440450.0,Information,592.0,87171.0
36,California,16.0,422364.0,Information,978.0,74674.0
38,California,18.0,267823.0,Information,269.0,9135.0
67,California,47.0,166059.0,Information,807.0,20228.0


#### concat 2018 & 2021 california subset

In [101]:
# concat two subset of california for 2018 & 2021
result = (pd.concat([california18, california21], axis = 1, join = 'outer')
          .drop(['State','State','118th Congressional District'], axis=1)
          .reset_index())

In [102]:
# drop unnecessary columns
result = result.drop(['index',' Annual Payroll ($1,000) '], axis = 1)

In [103]:
col_lis = list(result.columns)
col_lis

['AverageSalary_18',
 'NAICS Description',
 ' Number of Establishments ',
 ' Employment ',
 'AverageSalary_21',
 'NAICS Description',
 ' Number of Establishments ',
 ' Employment ']

#### cleaning of columns names & creating more easy to understand names.

In [104]:
# cleaning of columns names & creating more easy to understand names.
for i in range(8):
    if i == 0:
        col_lis[i] = col_lis[i].strip()
        print(col_lis[i])
        pass
    elif 1 <= i < 4:
        col_lis[i] = col_lis[i].strip() + '_18'
        print(col_lis[i])
    elif i == 4:
         print(col_lis[i])
         pass
    else:
        col_lis[i] = col_lis[i].strip() + '_21'
        print(col_lis[i])

AverageSalary_18
NAICS Description_18
Number of Establishments_18
Employment_18
AverageSalary_21
NAICS Description_21
Number of Establishments_21
Employment_21


In [105]:
col_lis = col_lis
result.columns = col_lis


In [106]:
# shows information industry's data in california for 2021
result.loc[result['NAICS Description_21'] == 'Information']

,AverageSalary_18,NAICS Description_18,Number of Establishments_18,Employment_18,AverageSalary_21,NAICS Description_21,Number of Establishments_21,Employment_21
1,152064.0,Information,499.0,7084.0,197057.0,Information,499.0,6460.0
9,124485.0,Utilities,8.0,297.0,193516.0,Information,418.0,8258.0
10,127277.0,Utilities,18.0,249.0,327267.0,Information,1865.0,91735.0
13,323630.0,Information,612.0,53189.0,248708.0,Information,351.0,18622.0
14,203946.0,Management of companies and enterprises,189.0,13823.0,440450.0,Information,592.0,87171.0
15,132767.0,Utilities,17.0,2261.0,422364.0,Information,978.0,74674.0
17,348970.0,Information,984.0,66275.0,267823.0,Information,269.0,9135.0
46,126773.0,Utilities,30.0,781.0,166059.0,Information,807.0,20228.0


In [107]:
result.loc[result['NAICS Description_21'] == 'Arts, entertainment, and  in recreation']

,AverageSalary_18,NAICS Description_18,Number of Establishments_18,Employment_18,AverageSalary_21,NAICS Description_21,Number of Establishments_21,Employment_21


In [108]:
result.loc[result['NAICS Description_18'] == 'Arts, entertainment, and recreation']

,AverageSalary_18,NAICS Description_18,Number of Establishments_18,Employment_18,AverageSalary_21,NAICS Description_21,Number of Establishments_21,Employment_21
32,162265.0,"Arts, entertainment, and recreation",4530.0,18533.0,120758.0,Utilities,17.0,1126.0


In [109]:
# code gives the total number of congessional districts where a particular industry pay max average salary for 2021.
result.groupby('NAICS Description_21')['Number of Establishments_21'].count()

NAICS Description_21
Agriculture, forestry, fishing and hunting           1
Arts, entertainment, and recreation                  4
Finance and insurance                                3
Industries not classified                            2
Information                                          8
Management of companies and enterprises              7
Mining, quarrying, and oil and gas extraction        3
Professional, scientific, and technical services     1
Utilities                                           23
Name: Number of Establishments_21, dtype: int64

In [110]:
# code gives the total number of congessional districts where a particular industry pay max average salary for 2018.
result.groupby('NAICS Description_18')['Number of Establishments_18'].count()

NAICS Description_18
Agriculture, forestry, fishing and hunting        2
Arts, entertainment, and recreation               1
Finance and insurance                             4
Information                                       5
Management of companies and enterprises           5
Mining, quarrying, and oil and gas extraction     3
Professional, scientific, and technical           2
Utilities                                        31
Name: Number of Establishments_18, dtype: int64

#### formatting dataframe

In [111]:
def highlight_cols(x):
      
    # copy df to new - original data is not changed
    result = x.copy()
      
    # select all values to green color
    result.loc[:, :] = 'background-color: cyan'
      
    # overwrite values grey color
    result[['AverageSalary_18','NAICS Description_18','Number of Establishments_18','Employment_18']] = 'background-color: yellow'
      
    # return color df
    return result 
  
print("Highlighted DataFrame :")

result.style.apply(highlight_cols, axis = None)


Highlighted DataFrame :


,AverageSalary_18,NAICS Description_18,Number of Establishments_18,Employment_18,AverageSalary_21,NAICS Description_21,Number of Establishments_21,Employment_21
0,144577.000000,Utilities,55.000000,1175.000000,151783.000000,Utilities,72.000000,1536.000000
1,152064.000000,Information,499.000000,7084.000000,197057.000000,Information,499.000000,6460.000000
2,124904.000000,Utilities,43.000000,1066.000000,156769.000000,Utilities,47.000000,2143.000000
3,132342.000000,Utilities,54.000000,1337.000000,142622.000000,Utilities,26.000000,1715.000000
4,148628.000000,Utilities,22.000000,1546.000000,125031.000000,Utilities,44.000000,454.000000
5,112070.000000,Utilities,21.000000,1379.000000,131545.000000,"Mining, quarrying, and oil and gas extraction",3.000000,22.000000
6,155359.000000,Utilities,11.000000,665.000000,126798.000000,Utilities,22.000000,1972.000000
7,105096.000000,Utilities,60.000000,1102.000000,167135.000000,Management of companies and enterprises,60.000000,4201.000000
8,144211.000000,Utilities,21.000000,4130.000000,113658.000000,Utilities,18.000000,228.000000
9,124485.000000,Utilities,8.000000,297.000000,193516.000000,Information,418.000000,8258.000000


#### performing web scrapping to get congessional districts name from web.

In [112]:
# web scrapping to get congessional districts name from web.
url = r'https://en.wikipedia.org/wiki/California%27s_congressional_districts'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')


In [115]:
#create 118th congressional districts name list with beautifulsoup module
cong_dis = []
for i in soup.find_all('span', style = True):
    cong_dis.append(i.text.replace('(','').replace(')',''))
cong_dis = cong_dis[2:55]
cong_dis



['Oroville',
 'San Rafael',
 'Rocklin',
 'St. Helena',
 'Elk Grove',
 'Elk Grove',
 'Sacramento',
 'Walnut Grove',
 'Tracy',
 'Concord',
 'San Francisco',
 'Oakland',
 'Modesto',
 'Livermore',
 'South San Francisco',
 'Atherton',
 'Fremont',
 'San Jose',
 'Carmel Valley',
 'Bakersfield',
 'Fresno',
 'Hanford',
 'Big Bear Lake',
 'Santa Barbara',
 'Indio',
 'Westlake Village',
 'Santa Clarita',
 'Monterey Park',
 'Los Angeles',
 'Burbank',
 'Norwalk',
 'Los Angeles',
 'Redlands',
 'Los Angeles',
 'Pomona',
 'Torrance',
 'Los Angeles',
 'Whittier',
 'Riverside',
 'Anaheim Hills',
 'Corona',
 'Long Beach',
 'Los Angeles',
 'Los Angeles',
 'Seal Beach',
 'Santa Ana',
 'Irvine',
 'San Diego',
 'San Juan Capistrano',
 'San Diego',
 'San Diego',
 'San Diego',
 '[permanent dead link]']

In [82]:

'''result['districts'] = soup.find_all('span', style = True)[2:]
result['districts'].astype(str)[0][-20:-12]
'''

"result['districts'] = soup.find_all('span', style = True)[2:]\nresult['districts'].astype(str)[0][-20:-12]\n"

#### create congressional districts name column & inserting at starting position in dataframe

In [116]:
# create congressional districts name column & inserting at starting position
result.insert(0, 'Cong_Dis', cong_dis)
result.index = result.index + 1
result.head(3)

,Cong_Dis,AverageSalary_18,NAICS Description_18,Number of Establishments_18,Employment_18,AverageSalary_21,NAICS Description_21,Number of Establishments_21,Employment_21
1,Oroville,144577.0,Utilities,55.0,1175.0,151783.0,Utilities,72.0,1536.0
2,San Rafael,152064.0,Information,499.0,7084.0,197057.0,Information,499.0,6460.0
3,Rocklin,124904.0,Utilities,43.0,1066.0,156769.0,Utilities,47.0,2143.0


In [84]:
result.loc[(result['Cong_Dis'] == 'Elk Grove')].AverageSalary_18.max()
(result.loc[(result['Cong_Dis'] == 'Elk Grove') & (result.AverageSalary_18 == result.loc[(result['Cong_Dis'] == 'Elk Grove')]
                                                   .AverageSalary_18.max())])


,Cong_Dis,AverageSalary_18,NAICS Description_18,Number of Establishments_18,Employment_18,AverageSalary_21,NAICS Description_21,Number of Establishments_21,Employment_21
5,Elk Grove,148628.0,Utilities,22.0,1546.0,125031.0,Utilities,44.0,454.0


In [85]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 1 to 53
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Cong_Dis                     53 non-null     object 
 1   AverageSalary_18             53 non-null     float64
 2   NAICS Description_18         53 non-null     object 
 3   Number of Establishments_18  53 non-null     float64
 4   Employment_18                53 non-null     float64
 5   AverageSalary_21             52 non-null     float64
 6   NAICS Description_21         52 non-null     object 
 7   Number of Establishments_21  52 non-null     float64
 8   Employment_21                52 non-null     float64
dtypes: float64(6), object(3)
memory usage: 3.9+ KB
